In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as mt
from metpy.interpolate import interpolate_to_points
import scipy.optimize as spo
import time 
from joblib import Parallel, delayed
import glob
from os import walk

In [2]:
# creating the pulse field

def gen_field(dur_max, dur_step):
    pdur_i0 = ppau_i0 = np.arange(0, dur_max, dur_step)
    pdur_j, ppau_j = np.meshgrid(pdur_i0, ppau_i0)
    return pdur_j, ppau_j


In [3]:
#Plotting the data

def plot_field(pdur, ppau, res):
    plt.pcolor(pdur, ppau, res, cmap='Reds')
    plt.xlim(0, np.max(pdur))
    plt.ylim(0, np.max(ppau))
    plt.colorbar(label='Phonotaxis')
    plt.xlabel('Pulse [ms]')
    plt.ylabel('Pause [ms]')
    plt.show()

In [4]:
#filtering data to have similar chirp structure and eliminating redundancies in pulse pause and duration

def filter_data(x, dur_max):
    pd_df = x
    
    #finding frequency of each chirp duration
    x = np.array(pd_df.CDUR)
    x = x.astype(int)
    (uniq, freq) = (np.unique(x, return_counts=True))
    xa = np.array([uniq,freq])

    #finding the bracket of 61 values with the most data points
    if (np.max(xa[0]) - np.min(xa[0])) > 60: #Checking if the range of available points exceeds interested bracket size
        jo = np.zeros(np.max(xa[0]))
        jo[xa[0]-1] = freq
        jo2 = []
        for i in range(30,len(jo)-30):
            jo2.append([i, np.sum(jo[i-30:i+30])])
        jo2 = np.array(jo2).T
        j = np.where(jo2 == np.max(jo2[1]))
        val = j[1][1] + 31

        #filtering down to values in the bracket calculated above
        x = pd_df.CDUR
        x1 = x>=(val-30)
        xa = pd_df[x1]
        x2 = xa.CDUR<=(val+30)
        xb = xa[x2]
    
    else: 
        xb = pd_df
    
    #repeating the same procedure for the values in chirp pause
    pd_df = xb
    x = np.array(pd_df.CPAU)
    x = x.astype(int)
    (uniq, freq) = (np.unique(x, return_counts=True))
    xa = np.array([uniq,freq])

    if (np.max(xa[0]) - np.min(xa[0])) > 60:
        jo = np.zeros(np.max(xa[0]))
        jo[xa[0]-1] = freq
        jo2 = []
        for i in range(30,len(jo)-30):
            jo2.append([i, np.sum(jo[i-30:i+30])])
        jo2 = np.array(jo2).T
        j = np.where(jo2 == np.max(jo2[1]))
        val = j[1][1] + 31

        x = pd_df.CPAU
        x1 = x>=(val-30)
        xa = pd_df[x1]
        x2 = xa.CPAU<=(val+30)
        xb1 = xa[x2]
    
    else: 
        xb1 = xb

    #averaging the duplicates
    dfa = pd.DataFrame([xb1.PDUR, xb1.PPAU, xb1.rXY]).T
    dfx = dfa.groupby(['PPAU', 'PDUR']).mean().reset_index()
    
    #filtering for values under the maximum duration specified
    check = dfx<=dur_max
    df_c = np.all(check,1)
    dfx = dfx[df_c]
    
    dfx = np.array([dfx.PDUR, dfx.PPAU, dfx.rXY]).T
    
    dur_max = np.max([np.max(dfx.T[0]), np.max(dfx.T[1])])
    print(dur_max)
    dur_max = int(dur_max*1.25)
    dur_steps = int(np.sqrt((dur_max**2 * 16)/2500))/4
    print(dur_steps)
    
    edg = [[0,0,0], [0, dur_max, 0], [dur_max, dur_max, 0], [dur_max,0,0]] #initializing origin to zero (for smooth interpolation purposes)
    dfx = np.append(dfx, edg, axis = 0) 
    dfx = dfx.T
    
    #plotting for verifying of data points
    plt.plot(dfx[0], dfx[1], 'o')
    plt.xlim(0, dur_max)
    plt.ylim(0, dur_max)
    plt.show()
    return dfx, dur_max, dur_steps


In [5]:
#Importing data from file

def import_data(species, dur_max):
    
    pd_df = species
    df, dur_max, dur_step = filter_data(pd_df, dur_max)    
    
    pdur = df[0]
    ppau = df[1]
    phonotaxis = df[2]

    np.random.seed(10)
    # need to jitter data points slightly for interp to work - probably a bug in metpy
    points = np.array([pdur + np.random.randn(*pdur.shape)/10000000, ppau + np.random.randn(*ppau.shape)/10000000]).T

    # make new grid of points to interpolate to
    pdur_i0 = ppau_i0 = np.arange(0, dur_max, dur_step)
    pdur_i, ppau_i = np.meshgrid(pdur_i0, ppau_i0)
    new_points = np.array([pdur_i, ppau_i]).T.reshape((-1, 2))

    # natural neighbour interpolation 
    ppf = interpolate_to_points(points, phonotaxis, new_points, interp_type='natural_neighbor')
    ppf[np.isnan(ppf)] = 0 #np.nanmean(ppf)
    ppf = np.maximum(ppf, 0)  # set neg vals to 0
    ppf = ppf.reshape((len(pdur_i), len(ppau_i))).T  # make interpolated ppf square
    
    ppf /= np.max(ppf)
    return ppf, dur_max, dur_step



In [6]:
#generating the signal from the parameter duty cycle

def signal(idur, ipau, sf):
    sf = int(sf)
    unit = (sf/1000)
    dur = int(unit*idur)
    pau = int(unit*ipau)
    #print(dur, pau, unit)
    adur = np.ones(dur, dtype = int)
    apau = np.zeros(pau, dtype = int)
    aper = np.concatenate((adur,apau))
    sig2 = np.tile(aper, sf)
    if len(sig2)>sf:
        sig = sig2[0:sf]
    else:
        l = sf - len(sig2)
        sig3 = np.zeros(l, dtype = int)
        sig = np.concatenate((sig2, sig3))
    return sig

In [7]:
#The Gabor filter

def gabor(fr, sigma, phi, w, sf):
    border = int((3.5*sigma)) 
    t =  np.arange(-border, border, 1000/sf) #in ms
    gaussian = np.exp(-(t)**2/(2*sigma**2)) # in ms
    sinusoidal = np.sin(2*np.pi*(fr/1000)*t + phi) # in KHz, ms
    gbr =  gaussian * sinusoidal + w
    return gbr


In [8]:
#linear nonlinear filtering (including the integration)

def lin_nonlin(sig, gab, a1, b1):
    f1 = np.convolve(sig, gab) #linear filter
    
    g1 = 1/(1 + np.exp( -(a1 * f1) + b1)) # nonlinear sigmoid function

    #integral (which is basically summation)
    v1 = 0.001 * g1.sum()
    return v1

In [9]:
#extracting Phonotaxis value by pushing fabricated signal through filter

def phonotaxis(fr, sigma, phi, w, a1, b1, sf, t1, t2):
    sig1 = signal(t1, t2, sf)
    gab1 = gabor(fr, sigma, phi, w, sf)
    phntxs = lin_nonlin(sig1, gab1, a1, b1)
    return phntxs


In [10]:
#the entire model as a single function which is to be minimized 
#the function calculates the difference between the generated pulse-pause preference data and the actual experimental data imported earlier in the program.

def sig_diff(cfg):
    fr, sigma, ph, w, a1, b1 = cfg #unpacking the command from gods
    phi = np.pi * ph

    #generate field
    sf = 2000
    
    #calculating phonotaxis values
    phono = np.vectorize(phonotaxis)
    pnt = phono(fr, sigma, phi, w, a1, b1, sf, pdur_i, ppau_i)
    
    #normalization
    pnt = np.maximum(pnt, 0)
    if (np.max(pnt) - np.min(pnt)) > 0:
        pnt = (pnt - np.min(pnt))/(np.max(pnt) - np.min(pnt))
    else:
        pnt = (pnt - np.min(pnt))
    #RMS of difference
    diff = np.sqrt(np.mean((pnt - pnt_ori)**2))
    
    return diff

In [11]:
#redundant function for crossverification and plotting

def sig_diff2(cfg):
    fr, sigma, ph, w, a1, b1 = cfg #unpacking the command from gods
    phi = np.pi * ph

    #generate field
    sf = 2000
    
    #calculating phonotaxis values
    phono = np.vectorize(phonotaxis)
    pnt = phono(fr, sigma, phi, w, a1, b1, sf, pdur_i, ppau_i)
    
    #normalization
    pnt = np.maximum(pnt, 0)
    if (np.max(pnt) - np.min(pnt)) > 0:
        pnt = (pnt - np.min(pnt))/(np.max(pnt) - np.min(pnt))
    else:
        pnt = (pnt - np.min(pnt))
    #RMS of difference
    diff = np.sqrt(np.mean((pnt - pnt_ori)**2))
    
    return diff, pnt

In [12]:
# Example parameters for signal generation
# sf = 2000

# #PARAMETERS OF GABOR FUNCTION
# #frequency for the Gabor filter - INFLUENCES pulse period preference
# fr = 50 #in Hz
# #sharpness of tuning for pulse period
# sigma = 50 
# #phase shift - change the integer to change the phase 
# phi = np.pi * 0 
# #offset - INFLUENCES duty cycle preference
# w = 0

# #parameters for Lin_nonlinear
# a1 = 0.05 #slope/steepness of sigmoid
# b1 = 1 #1/2 of max of non linearity

# #pulse pause field sampling range and frequency(step)
# dur_max = 20  # ms
# dur_step = 0.5  # ms

In [13]:
# Extract all files in folder

def file_extract():
    files = glob.glob('dat/Xls/*.csv')
    data = []
    for file in files:
        data.append(pd.read_csv(file))
    file_no = len(data)
    # for i in range(file_no)
    filename = files
    filename = [s.replace('dat/Xls/', '').replace('_ppf.csv', '') for s in filename] 
    kou = [filename, data] #knowledge of universe
    return kou 


In [21]:
def best_fit(cfg, temp):

    #setting predetermined initial parameters, evaluating the start point of minimize function and plotting the ppf corresponding to the initial parameters\
    print(cfg)
    y = sig_diff(cfg)
    print(y)
    xz = sig_diff2(cfg)
    plot_field(pdur_i, ppau_i, xz[1])

    #bounds of each parameter

    b_fr = (0, 700)
    b_sigma = (1, 501)
    b_phi = (0, 2)
    b_w = (-1, 1)
    b_a1 = (0, np.inf)
    b_b1 = (-np.inf, np.inf)
    bnds = (b_fr, b_sigma, b_phi, b_w, b_a1, b_b1)
    
    # Basin hopping minimization - to find the closest fit with the existing data without getting stuck in a local minima. 'options' : {"disp": True },

    result = spo.basinhopping(sig_diff, cfg, niter = 100, T=temp, minimizer_kwargs={ 'bounds' : bnds,  'method' : 'L-BFGS-B'}) 

    if result.fun >= 0.12:
        temp = temp+0.01
        best_fit(cfg, temp)
    
    print(result)
    xe = sig_diff2(result.x)
    plot_field(pdur_i, ppau_i, xe[1])
    
    return result


In [22]:
# Extract all files in folder

def file_extract():
    files = glob.glob('dat/Xls/*.csv')
    data = []
    for file in files:
        data.append(pd.read_csv(file))
    file_no = len(data)
    # for i in range(file_no)
    filename = files
    filename = [s.replace('dat/Xls/', '').replace('_ppf.csv', '') for s in filename] 
    kou = [filename, data] #knowledge of universe
    return filename, data 


In [23]:
def func_call(i):
    #predetermined list of initial parameters
    cfg_list = [[13, 22, 1.4, -0.08, 0.23, 5], [130, 7, 1.4, 0.0, 0.15, 5],[60, 20, 1.4, 0.0, 0.15, 5],[28, 20, 1.4, -0.02, 0.19, 5],[18, 32, 0.5, -0.003, 0.09, 15], [16, 15, 0.5, -0.2, 0.13, 29], [30, 17, 0.5, 0.0, 0.04, 29],[90, 4, 1, 0.0, 0.001, 29],[70, 17, 1, 0.0, 0.1, 29], [65, 10, 0, 0.0, 0.01, 8], [12, 120, 1.1, 0.0, 0.0001, 3], [30, 35, 1.5, -0.002, 0.01, 30], [60, 2, 0, 0.0, 0.2, 30], [60, 2, 0, 0.0, 0.2, 30], [60, 15, 0, 0.0, 0.01, 10], [28, 15, 0.5, -0.02, 0.09, 52], [30, 20, 0.9, -0.02, 0.09, 52], [30, 20, 1.3, -0.04, 0.09, 10], [30, 19, 1.3, -0.04, 0.15, 10]]
    cfg_name = ['ADO', 'ANU', 'ARM', 'BIM', 'FIR', 'G13', 'G14', 'G15', 'GSP', 'LIN', 'LOC', 'OVI', 'PER', 'RUB', 'TEX', 'TUL', 'VEL', 'VOX', 'YUC']
    cfg_array = np.array(cfg_list)
    
    dataname, data = file_extract()
    del data[11]
    del dataname[11]
    
    temp = 0.01
    
    # all_results = []
    
    dur_max = 80  # ms
    global pnt_ori, pdur_i, ppau_i
    unpack = import_data(data[i], dur_max)
    pnt_ori = unpack[0]
    dur_max = unpack[1] #in ms
    dur_step = unpack[2] #in ms
    pnt_ori /= np.max(pnt_ori)
    pdur_i, ppau_i = gen_field(dur_max, dur_step)

    #Plot original datapoints for reference
    plot_field(pdur_i, ppau_i, pnt_ori)

    cfg = cfg_list[i]

    print(cfg_name[i])

    res = best_fit(cfg, temp)
    xe = sig_diff2(res.x)
    plot_field(pdur_i, ppau_i, xe[1])
    # all_results.append(res)
        
    return res
        
        

In [24]:
def main():
    all_res = Parallel(n_jobs=10)(delayed(func_call)(i) for i in range(19))
    return all_res


In [ ]:
main()

/tmp/ipykernel_10542/3250708937.py:40: RuntimeWarning: overflow encountered in long_scalars
/tmp/ipykernel_10542/3250708937.py:40: RuntimeWarning: overflow encountered in long_scalars
/tmp/ipykernel_10542/901067433.py:6: RuntimeWarning: overflow encountered in exp
/tmp/ipykernel_10542/901067433.py:6: RuntimeWarning: overflow encountered in exp
/tmp/ipykernel_10542/901067433.py:6: RuntimeWarning: overflow encountered in exp
/tmp/ipykernel_10542/901067433.py:6: RuntimeWarning: overflow encountered in exp
/tmp/ipykernel_10542/901067433.py:6: RuntimeWarning: overflow encountered in exp
